# Attempt 3:
## Target: 
<TBD>

## Results:
<TBD>

## Analysis
<TBD>

# Import Libraries

In [ ]:
# importing all the Python Packages & torch Library.
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Define Data Transformations & Dataset for Train/Test

In [ ]:
train_transforms = transforms.Compose([
                        transforms.RandomRotation((-10.0, 10.0), fill=(1,)),
                        # transforms.RandomErasing(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

test_transforms = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

# Dataloader Arguments & Train / Test Dataloaders

In [ ]:
# seed the model to obtain consistent results
torch.manual_seed(1)
# this is the batch size , in 1 pas no of images passed together.
batch_size = 64
use_cuda = torch.cuda.is_available()

print("CUDA Available?", use_cuda)

# kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if use_cuda else dict(shuffle=True, batch_size=64)



# load the training data and perform standard normalization 
# parameter for normalization is mean and std dev.
# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

# The Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(3, 3), padding=0, bias = True), # output 26X26X4 : RF- 3X3
            nn.ReLU(),
            nn.BatchNorm2d(4), # Batch Normalization after each convolution.
            nn.Dropout2d(0.08), # dropout of 10% at each layer
            
            nn.Conv2d(4, 8, 3), # output 24X24X8 : RF- 5x5
            nn.ReLU(),
            nn.BatchNorm2d(8),  # Batch Normalization after each convolution.
            nn.Dropout2d(0.08),  # dropout of 10% at each layer

            nn.Conv2d(8, 8, 3), # output 22X22X16 : RF- 7X7
            nn.ReLU(),
            nn.BatchNorm2d(8),  # Batch Normalization after each convolution.
            nn.Dropout2d(0.08),  # dropout of 10% at each layer

            nn.MaxPool2d(2, 2)       # output 11X11X16 : RF- 
            )
        self.conv2 = nn.Sequential(
            
           
            nn.Conv2d(8, 16, 3), # output 9X9X16 : RF- 18 X18
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.08),

            nn.Conv2d(16, 8, 3), # output 9X9X16 : RF- 18 X18
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.08),

            )
        self.conv3 = nn.Sequential(
           
            nn.Conv2d(8, 16, 3,padding=1), # output 07X07X16 : RF- 22 X 22
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.08),

            nn.Conv2d(16, 10, 1), # output 05X05X10 : RF- 22 X 22
            nn.AvgPool2d(7)

            )
        
    def forward(self, x):
        
        x = self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, -1)

# Print Summary of Model

In [ ]:
!pip install torchsummary
from torchsummary import summary
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# Define Train and Test functions

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

# Function to train 
'''
Args: 
Model : created model to be used for training
device : GPU or cpu
train_laoded: data on which the training has to be done
Optimizer : the optimization algorithm to be used
epoch : no fo epoch 

'''
def train(model, device, train_loader, optimizer, epoch):
    model.train() # Set the model on training mode
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # moving the data to device
        optimizer.zero_grad() # zero the graidents 
        output = model(data) # getting the model output
        loss = F.nll_loss(output, target) # calculating the The negative log likelihood loss
        train_losses.append(loss)
        loss.backward() # flowing the gradients backward.
        optimizer.step() # paameter updated basd on the current gradient.
        
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)        
        
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

# Function to test 
'''
Args: 
Model : created model to be used for training
device : GPU or cpu
test_laoded: data on which the testing has to be done
 

'''
def test(model, device, test_loader):
    model.eval() # seting up the model for evalaution.
    test_loss = 0 # setting the test loss to 0
    correct = 0 # countign the no of coorect classfication.
    with torch.no_grad(): # turn off gradients, since we are in test mode
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)   # copy the data to device.
            output = model(data) # predict the output
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset) # calculating hte test loss.
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Test Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc.append(100. * correct / len(test_loader.dataset))

# Run the model

In [ ]:
from torch.optim.lr_scheduler import StepLR


model = Net().to(device) # move the model to device.
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)  # intiating the SGD optimizer
scheduler = StepLR(optimizer, step_size=6, gamma=0.5)


for epoch in range(1, 16):
    print("epoch =", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)
    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot(train_losses)
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(train_acc)
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")